# Constrained and Guided Generation Tutorial
### Overview
This tutorial explores the concepts of constrained and guided generation in the context of large language models. We'll focus on techniques to set up constraints for model outputs and implement rule-based generation using Gemini models and the LangChain library.

### Motivation
While large language models are powerful tools for generating text, they sometimes produce outputs that are too open-ended or lack specific desired characteristics. Constrained and guided generation techniques allow us to exert more control over the model's outputs, making them more suitable for specific tasks or adhering to certain rules and formats.

### Key Components
1. Setting up constraints for model outputs
2. Implementing rule-based generation
3. Using LangChain's PromptTemplate for structured prompts
4. Leveraging Gemini models for text generation
### Method Details
We'll use a combination of prompt engineering techniques and LangChain's utilities to implement constrained and guided generation:

1. We'll start by setting up the environment and importing necessary libraries.
2. We'll create structured prompts using LangChain's PromptTemplate to guide the model's output.
3. We'll implement constraints by specifying rules and formats in our prompts.
4. We'll use Gemini model to generate text based on our constrained prompts.
5. We'll explore different techniques for rule-based generation, including output parsing and regex-based validation.
### Conclusion
By the end of this tutorial, you'll have a solid understanding of how to implement constrained and guided generation techniques. These skills will enable you to create more controlled and specific outputs from large language models, making them more suitable for a wide range of applications where precise and rule-adherent text generation is required.

### Setup
First, let's import the necessary libraries and set up our environment.

In [2]:
import os
import re
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.output_parsers import RegexParser

# Load enviroment variables
from dotenv import load_dotenv
load_dotenv()

# Set up Google API key
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

# Initialize the language model
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

# Function to display model outputs
def display_output(output):
    """Display the model's output in a formatted manner."""
    print("Model Output:")
    print("-" * 40)
    print(output)
    print("-" * 40)
    print()

### Setting Up Constraints for Model Outputs
Let's start by creating a constrained prompt that generates a product description with specific requirements.

In [3]:
constrained_prompt = PromptTemplate(
    input_variables=["product", "target_audience", "tone", "word_limit"],
    template="""Create a product description for {product} targeted at {target_audience}.
    Use a {tone} tone and keep it under {word_limit} words.
    The description should include:
    1. A catchy headline
    2. Three key features
    3. A call to action
    
    Product Description:
    """
)

# Generate the constrained output
input_variables = {
    "product": "smart water bottle",
    "target_audience": "health-conscious millennials",
    "tone": "casual and friendly",
    "word_limit": "75"
}

chain = constrained_prompt | llm
output = chain.invoke(input_variables).content
display_output(output)

Model Output:
----------------------------------------
**Hydration, Simplified: Meet Your New Smart Bottle!**

Stay on top of your H2O game with our sleek smart bottle! It tracks your water intake, glows to remind you to drink, and even syncs with your fitness tracker. Plus, it's made with sustainable materials because, you know, #eco-friendly. Ready to level up your hydration? Grab yours now and let's glow!
----------------------------------------



### Implementing Rule-Based Generation
Now, let's implement a rule-based generation system for creating structured job postings.

In [4]:
job_posting_prompt = PromptTemplate(
    input_variables=["job_title", "company", "location", "experience"],
    template="""Create a job posting for a {job_title} position at {company} in {location}.
    The candidate should have {experience} years of experience.
    Follow these rules:
    1. Start with a brief company description (2 sentences)
    2. List 5 key responsibilities, each starting with an action verb
    3. List 5 required qualifications, each in a single sentence
    4. End with a standardized equal opportunity statement
    
    Format the output as follows:
    COMPANY: [Company Description]
    
    RESPONSIBILITIES:
    - [Responsibility 1]
    - [Responsibility 2]
    - [Responsibility 3]
    - [Responsibility 4]
    - [Responsibility 5]
    
    QUALIFICATIONS:
    - [Qualification 1]
    - [Qualification 2]
    - [Qualification 3]
    - [Qualification 4]
    - [Qualification 5]
    
    EEO: [Equal Opportunity Statement]
    """
)

# Generate the rule-based output
input_variables = {
    "job_title": "Senior Software Engineer",
    "company": "TechInnovate Solutions",
    "location": "San Francisco, CA",
    "experience": "5+"
}

chain = job_posting_prompt | llm
output = chain.invoke(input_variables).content
display_output(output)

Model Output:
----------------------------------------
COMPANY: TechInnovate Solutions is a leading provider of innovative software solutions for the financial technology industry. We are committed to building cutting-edge products and fostering a collaborative, growth-oriented environment for our employees.

RESPONSIBILITIES:
- Design, develop, and maintain robust and scalable software applications.
- Collaborate with product managers and other engineers to define and implement new features.
- Write clean, well-documented, and testable code.
- Participate in code reviews to ensure code quality and share knowledge.
- Troubleshoot and resolve complex technical issues in a timely manner.

QUALIFICATIONS:
- You possess a Bachelor's degree in Computer Science or a related field.
- You have 5+ years of experience in software development.
- You are proficient in at least one of the following languages: Java, Python, or Go.
- You have experience with relational databases and NoSQL databases.


### Using Regex Parser for Structured Output
Let's use a regex parser to ensure our output adheres to a specific structure.

In [5]:
# Define a regex parser for structured output
regex_parser = RegexParser(
    regex=r"COMPANY:\s*([\s\S]*?)\n\s*RESPONSIBILITIES:\s*([\s\S]*?)\n\s*QUALIFICATIONS:\s*([\s\S]*?)\n\s*EEO:\s*([\s\S]*)",
    output_keys=["company_description", "responsibilities", "qualifications", "eeo_statement"]
)
# This regex pattern captures the company description, responsibilities, qualifications, and EEO statement from the output text.

# Create a new prompt template that includes the parser instructions
parsed_job_posting_prompt = PromptTemplate(
    input_variables=["job_title", "company", "location", "experience"],
    template="""Create a job posting for a {job_title} position at {company} in {location}.
    The candidate should have {experience} years of experience.
    Follow these rules:
    1. Start with a brief company description (2 sentences)
    2. List 5 key responsibilities, each starting with an action verb
    3. List 5 required qualifications, each in a single sentence
    4. End with a standardized equal opportunity statement
    
    Format the output EXACTLY as follows:
    COMPANY: [Company Description]
    
    RESPONSIBILITIES:
    - [Responsibility 1]
    - [Responsibility 2]
    - [Responsibility 3]
    - [Responsibility 4]
    - [Responsibility 5]
    
    QUALIFICATIONS:
    - [Qualification 1]
    - [Qualification 2]
    - [Qualification 3]
    - [Qualification 4]
    - [Qualification 5]
    
    EEO: [Equal Opportunity Statement]
    """
)

def clean_output(output):
    for key, value in output.items():
        if isinstance(value, str):
            # Remove leading/trailing whitespace and normalize newlines
            output[key] = re.sub(r'\n\s*', '\n', value.strip())
    return output

# Generate the parsed output
chain = parsed_job_posting_prompt | llm
raw_output = chain.invoke(input_variables).content

# Parse and clean the output
parsed_output = regex_parser.parse(raw_output)
cleaned_output = clean_output(parsed_output)

# Display the parsed output
print("Parsed Output:")
for key, value in cleaned_output.items():
    print(f"{key.upper()}:")
    print(value)
    print()

Parsed Output:
COMPANY_DESCRIPTION:
TechInnovate Solutions is a leading technology company committed to developing innovative solutions for a wide range of industries. We foster a collaborative and dynamic work environment where employees can thrive and make a significant impact.

RESPONSIBILITIES:
- Design and develop high-quality, scalable, and maintainable software solutions.
- Collaborate with cross-functional teams to define, design, and ship new features.
- Participate in code reviews to ensure code quality and share knowledge.
- Troubleshoot and debug complex software issues in a timely manner.
- Contribute to the continuous improvement of our development processes and tools.

QUALIFICATIONS:
- You possess a Bachelor's degree in Computer Science or a related field.
- You have 5+ years of experience in software development.
- You are proficient in at least one programming language such as Java, Python, or C++.
- You have experience with cloud technologies such as AWS, Azure, or G

### Implementing Additional Constraints
Let's create a more complex constrained generation task: generating a product review with specific criteria.

In [6]:
review_prompt = PromptTemplate(
    input_variables=["product", "rating", "pros", "cons", "word_limit"],
    template="""Write a product review for {product} with the following constraints:
    1. The review should have a {rating}-star rating (out of 5)
    2. Include exactly {pros} pros and {cons} cons
    3. Use between 2 and 3 sentences for each pro and con
    4. The entire review should be under {word_limit} words
    5. End with a one-sentence recommendation
    
    Format the review as follows:
    Rating: [X] out of 5 stars
    
    Pros:
    1. [Pro 1]
    2. [Pro 2]
    ...
    
    Cons:
    1. [Con 1]
    2. [Con 2]
    ...
    
    Recommendation: [One-sentence recommendation]
    """
)

# Generate the constrained review
input_variables = {
    "product": "Smartphone X",
    "rating": "4",
    "pros": "3",
    "cons": "2",
    "word_limit": "200"
}

chain = review_prompt | llm
output = chain.invoke(input_variables).content
display_output(output)

Model Output:
----------------------------------------
Rating: 4 out of 5 stars

Pros:
1. Excellent Camera: The camera on the Smartphone X is truly impressive, capturing sharp and vibrant photos even in low-light conditions. The image stabilization is top-notch, resulting in clear videos and stunning pictures.
2. Long Battery Life: I can easily get through a full day of heavy use without needing to reach for a charger. This is a huge plus for anyone who is constantly on the go and doesn't want to worry about battery anxiety.
3. Sleek Design: The phone boasts a modern and stylish design that feels premium in hand. The slim profile and comfortable grip make it a pleasure to use.

Cons:
1. Software Bloat: The phone comes pre-loaded with a considerable amount of bloatware that can be annoying to remove. It takes up valuable storage space and can sometimes impact performance, which is frustrating.
2. Average Speaker Quality: The speaker quality is adequate for calls and casual listening, bu